# Random Walk Generation

## Generate Graph from Sentence 

In [1]:
import networkx as nx
import pandas as pd
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")
import spacy
import scispacy

In [3]:
nlp = spacy.load("en_core_sci_lg")

In [4]:
PATH = "/Users/apple/Google Drive/_My Data Analytics Exercise/Exercise/Cytoscape/KEGG enzymes/"

In [5]:
df = pd.read_csv(PATH+"enzyme_sentence.csv")

In [6]:
df.head()

,enzyme,sentence
0,L-glutamate,L-glutamate catalyses ATP and L-cysteine to pr...
1,L-serine,L-serine catalyses ATP and L-glutamate and L-c...
2,"alpha-D-glucose 1,6-phosphomutase","alpha-D-glucose 1,6-phosphomutase catalyses AT..."
3,acetoacetate,acetoacetate catalyses ATP and L-glutamate and...
4,L-proline,L-proline catalyses ATP and L-glutamate and L-...


In [7]:
sent = df['sentence'][0]
enzyme = df['enzyme'][0]
print (sent)

L-glutamate catalyses ATP and L-cysteine to produce ADP and phosphate and gamma-L-glutamyl-L-cysteine


In [8]:
doc = nlp(sent)

In [9]:
for token in doc:
    print (token.text, token.pos_)
print ()
print ("Number of tokens: %i" %len(doc))

L-glutamate NOUN
catalyses VERB
ATP NOUN
and CCONJ
L-cysteine NOUN
to PART
produce VERB
ADP NOUN
and CCONJ
phosphate NOUN
and CCONJ
gamma-L-glutamyl-L-cysteine NOUN

Number of tokens: 12


In [10]:
from spacy import displacy
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [10]:
# Generate a list of nodes with arbitrarily assigned weight
nodes_list = dict()
for token in doc:
    if token.pos_ == "VERB" or token.text.lower() == "and" or token.text.lower() == "or" or token.text.lower() == 'not':
        nodes_list[token.text] = (token.pos_, 3)
    elif token.pos_ == "NOUN":
        nodes_list[token.text] = (token.pos_, 2)
    else:
        nodes_list[token.text] = (token.pos_, 1)

In [11]:
print (nodes_list)
print (len(set(nodes_list)))

{'L-glutamate': ('NOUN', 2), 'catalyses': ('VERB', 3), 'ATP': ('NOUN', 2), 'and': ('CCONJ', 3), 'L-cysteine': ('NOUN', 2), 'to': ('PART', 1), 'produce': ('VERB', 3), 'ADP': ('NOUN', 2), 'phosphate': ('NOUN', 2), 'gamma-L-glutamyl-L-cysteine': ('NOUN', 2)}
10


In [12]:
import string
exclude = set(string.punctuation)

In [13]:
# Building links among tokens
head = list()
tail = list()
for i, token in enumerate(doc):
    if token.n_lefts>=1:
        for t in token.lefts:
            if t.text in exclude:
                pass
            else:
                head.append(t.text)
                tail.append(token.text)
                head.append(token.text)
                tail.append(t.text)
    if token.n_rights>=1:
        for t in token.rights:
            if t.text in exclude:
                pass
            else:
                head.append(t.text)
                tail.append(token.text)
                head.append(token.text)
                tail.append(t.text)
    if token.text.lower() == "and" or token.text.lower() == "or":  # to create a triplet/clique of the network
        head.append(doc[i-1].text)
        tail.append(doc[i+1].text)
        head.append(doc[i+1].text)
        tail.append(doc[i-1].text)
        head.append(token.text)
        tail.append(doc[i-1].text)
        head.append(token.text)
        tail.append(doc[i+1].text)
    if token.text.lower() == 'not': # to conect two nearby nouns of which one negate the other
        #print (i, token.text)
        backward = -1
        while doc[i+backward].pos_ != "NOUN":
            backward -= 1
        #print (backward)
        start_token = doc[i+backward].text
        #print (start_token)
        forward = 1
        while doc[i+forward].pos_ != "NOUN":
            forward += 1
        #print (forward)
        end_token = doc[i+forward].text
        #print (end_token)
        head.append(start_token)
        tail.append('not')
        head.append('not')
        tail.append(end_token)

In [14]:
print (len(head), len(tail))

34 34


In [15]:
import pandas as pd
data = zip(head, tail)
df = pd.DataFrame(data,columns=['head','tail'])
pd.set_option("display.max_rows", 200, "display.max_columns", 6)
df

,head,tail
0,L-glutamate,catalyses
1,catalyses,L-glutamate
2,ATP,catalyses
3,catalyses,ATP
4,produce,catalyses
5,catalyses,produce
6,and,ATP
7,ATP,and
8,L-cysteine,ATP
9,ATP,L-cysteine


In [16]:
# Introducion of Edge Weight to highlight action between subject, verb and object
df['edge_wt'] = 1
for i in range(len(df)):
    head_node = df.loc[i, 'head']
    end_node = df.loc[i, 'tail']
    if (nodes_list[head_node][0]=="NOUN" and nodes_list[end_node][0]=="VERB"):
        df.loc[i,'edge_wt'] += 2
    if (nodes_list[head_node][0]=="VERB" and nodes_list[end_node][0]=="NOUN"):
        df.loc[i,'edge_wt'] += 2

In [17]:
print(nodes_list[df.loc[0,'head']][0] == 'NOUN')
print(df.loc[0,'head'])

print(nodes_list[df.loc[0,'tail']][0] == 'VERB')
print(df.loc[0,'tail'])

True
L-glutamate
True
catalyses


In [18]:
df.head()

,head,tail,edge_wt
0,L-glutamate,catalyses,3
1,catalyses,L-glutamate,3
2,ATP,catalyses,3
3,catalyses,ATP,3
4,produce,catalyses,1


In [19]:
tuples = list()
for i in range(len(df)):
    tuples.append((df.loc[i, 'head'], df.loc[i, 'tail'], df.loc[i, 'edge_wt']))

In [20]:
print (tuples, len(tuples))

[('L-glutamate', 'catalyses', 3), ('catalyses', 'L-glutamate', 3), ('ATP', 'catalyses', 3), ('catalyses', 'ATP', 3), ('produce', 'catalyses', 1), ('catalyses', 'produce', 1), ('and', 'ATP', 1), ('ATP', 'and', 1), ('L-cysteine', 'ATP', 1), ('ATP', 'L-cysteine', 1), ('ATP', 'L-cysteine', 1), ('L-cysteine', 'ATP', 1), ('and', 'ATP', 1), ('and', 'L-cysteine', 1), ('to', 'produce', 1), ('produce', 'to', 1), ('ADP', 'produce', 3), ('produce', 'ADP', 3), ('and', 'ADP', 1), ('ADP', 'and', 1), ('phosphate', 'ADP', 1), ('ADP', 'phosphate', 1), ('and', 'ADP', 1), ('ADP', 'and', 1), ('gamma-L-glutamyl-L-cysteine', 'ADP', 1), ('ADP', 'gamma-L-glutamyl-L-cysteine', 1), ('ADP', 'phosphate', 1), ('phosphate', 'ADP', 1), ('and', 'ADP', 1), ('and', 'phosphate', 1), ('phosphate', 'gamma-L-glutamyl-L-cysteine', 1), ('gamma-L-glutamyl-L-cysteine', 'phosphate', 1), ('and', 'phosphate', 1), ('and', 'gamma-L-glutamyl-L-cysteine', 1)] 34


In [21]:
print (len(set(tuples)))

25


In [22]:
connection = list(set(tuples))

In [23]:
print (connection)

[('catalyses', 'ATP', 3), ('catalyses', 'L-glutamate', 3), ('ADP', 'and', 1), ('and', 'ATP', 1), ('ATP', 'and', 1), ('ATP', 'catalyses', 3), ('produce', 'ADP', 3), ('gamma-L-glutamyl-L-cysteine', 'ADP', 1), ('produce', 'to', 1), ('and', 'L-cysteine', 1), ('produce', 'catalyses', 1), ('to', 'produce', 1), ('ATP', 'L-cysteine', 1), ('phosphate', 'ADP', 1), ('L-glutamate', 'catalyses', 3), ('and', 'phosphate', 1), ('catalyses', 'produce', 1), ('phosphate', 'gamma-L-glutamyl-L-cysteine', 1), ('gamma-L-glutamyl-L-cysteine', 'phosphate', 1), ('L-cysteine', 'ATP', 1), ('ADP', 'phosphate', 1), ('and', 'ADP', 1), ('and', 'gamma-L-glutamyl-L-cysteine', 1), ('ADP', 'produce', 3), ('ADP', 'gamma-L-glutamyl-L-cysteine', 1)]


In [24]:
import networkx as nx

In [25]:
G = nx.DiGraph()

In [26]:
# Adding weighted nodes to the graph
for node, values in nodes_list.items():
    G.add_node(node, weight=values[1])

In [27]:
G.nodes

NodeView(('L-glutamate', 'catalyses', 'ATP', 'and', 'L-cysteine', 'to', 'produce', 'ADP', 'phosphate', 'gamma-L-glutamyl-L-cysteine'))

In [28]:
# Adding weighted edge to the graph
for edge in connection:
    G.add_edge(edge[0], edge[1], weight=edge[2])

In [29]:
G.edges

OutEdgeView([('L-glutamate', 'catalyses'), ('catalyses', 'ATP'), ('catalyses', 'L-glutamate'), ('catalyses', 'produce'), ('ATP', 'and'), ('ATP', 'catalyses'), ('ATP', 'L-cysteine'), ('and', 'ATP'), ('and', 'L-cysteine'), ('and', 'phosphate'), ('and', 'ADP'), ('and', 'gamma-L-glutamyl-L-cysteine'), ('L-cysteine', 'ATP'), ('to', 'produce'), ('produce', 'ADP'), ('produce', 'to'), ('produce', 'catalyses'), ('ADP', 'and'), ('ADP', 'phosphate'), ('ADP', 'produce'), ('ADP', 'gamma-L-glutamyl-L-cysteine'), ('phosphate', 'ADP'), ('phosphate', 'gamma-L-glutamyl-L-cysteine'), ('gamma-L-glutamyl-L-cysteine', 'ADP'), ('gamma-L-glutamyl-L-cysteine', 'phosphate')])

In [30]:
# Testing
print(G.nodes['L-glutamate'])
print(G['L-glutamate']['catalyses'])

{'weight': 2}
{'weight': 3}


In [31]:
print (G['catalyses'])

{'ATP': {'weight': 3}, 'L-glutamate': {'weight': 3}, 'produce': {'weight': 1}}


## Predict and add new links to "VERB" using preferential attachment

In [32]:
# Generate additional link(s) based on path 2-step away from a "VERB"
verb = 'catalyses'
subgraph = nx.ego_graph(G, verb , radius=2)

In [33]:
two_step_away_neighbours = subgraph.nodes
print (subgraph.nodes)

['L-glutamate', 'catalyses', 'ATP', 'and', 'L-cysteine', 'to', 'produce', 'ADP']


In [34]:
# Generate pair list based on subgraph around the center word, i.e. VERB
node_pairs = list()
for node in two_step_away_neighbours:
    if node == verb or nodes_list[node][0] != "NOUN":
        pass
    else:
        node_pairs.append((verb, node))
print (node_pairs)

[('catalyses', 'L-glutamate'), ('catalyses', 'ATP'), ('catalyses', 'L-cysteine'), ('catalyses', 'ADP')]


In [35]:
G_undirected = G.to_undirected(as_view=True)

In [36]:
preds = nx.jaccard_coefficient(G_undirected, node_pairs)

In [37]:
for u, v, p in preds:
    print(f"({u}, {v}) -> {p}") 
# Largest value indicates preferential attachment

(catalyses, L-glutamate) -> 0.0
(catalyses, ATP) -> 0.0
(catalyses, L-cysteine) -> 0.25
(catalyses, ADP) -> 0.16666666666666666


In [38]:
for u, v, p in preds:
    if p==0:
        pass
    else:
        G.add_edge(u,v,weight=3)
        G.add_edge(v,u,weight=3)

In [39]:
verb = 'produce'
subgraph = nx.ego_graph(G, verb , radius=2)

In [40]:
two_step_away_neighbours = subgraph.nodes
print (subgraph.nodes)

['L-glutamate', 'catalyses', 'ATP', 'and', 'to', 'produce', 'ADP', 'phosphate', 'gamma-L-glutamyl-L-cysteine']


In [41]:
node_pairs = list()
for node in two_step_away_neighbours:
    if node == verb or nodes_list[node][0] != "NOUN":
        pass
    else:
        node_pairs.append((verb, node))
print (node_pairs)

[('produce', 'L-glutamate'), ('produce', 'ATP'), ('produce', 'ADP'), ('produce', 'phosphate'), ('produce', 'gamma-L-glutamyl-L-cysteine')]


In [42]:
preds = nx.jaccard_coefficient(G_undirected, node_pairs)
new_edges = list()
for u, v, p in preds:
    print(f"({u}, {v}) -> {p}") 
    new_edges.append((u,v))

(produce, L-glutamate) -> 0.3333333333333333
(produce, ATP) -> 0.2
(produce, ADP) -> 0.0
(produce, phosphate) -> 0.2
(produce, gamma-L-glutamyl-L-cysteine) -> 0.2


In [43]:
for u, v, p in preds:
    if p==0:
        pass
    else:
        G.add_edge(u,v,weight=3)
        G.add_edge(v,u,weight=3)

In [44]:
nx.write_gpickle(G, "sample_enzyme.gpickle")

## Generate Random Walk from Sentence Graph 

In [45]:
G = nx.read_gpickle("sample_enzyme.gpickle")

In [46]:
l = [n for n in G.neighbors('catalyses')]

In [47]:
type(len(l))

int

In [48]:
print (l)

['ATP', 'L-glutamate', 'produce']


In [49]:
node_weights = nx.get_node_attributes(G, "weight")

In [50]:
print (node_weights)

{'L-glutamate': 2, 'catalyses': 3, 'ATP': 2, 'and': 3, 'L-cysteine': 2, 'to': 1, 'produce': 3, 'ADP': 2, 'phosphate': 2, 'gamma-L-glutamyl-L-cysteine': 2}


In [51]:
edge_weights = nx.get_edge_attributes(G, 'weight')

In [52]:
print (edge_weights)

{('L-glutamate', 'catalyses'): 3, ('catalyses', 'ATP'): 3, ('catalyses', 'L-glutamate'): 3, ('catalyses', 'produce'): 1, ('ATP', 'and'): 1, ('ATP', 'catalyses'): 3, ('ATP', 'L-cysteine'): 1, ('and', 'ATP'): 1, ('and', 'L-cysteine'): 1, ('and', 'phosphate'): 1, ('and', 'ADP'): 1, ('and', 'gamma-L-glutamyl-L-cysteine'): 1, ('L-cysteine', 'ATP'): 1, ('to', 'produce'): 1, ('produce', 'ADP'): 3, ('produce', 'to'): 1, ('produce', 'catalyses'): 1, ('ADP', 'and'): 1, ('ADP', 'phosphate'): 1, ('ADP', 'produce'): 3, ('ADP', 'gamma-L-glutamyl-L-cysteine'): 1, ('phosphate', 'ADP'): 1, ('phosphate', 'gamma-L-glutamyl-L-cysteine'): 1, ('gamma-L-glutamyl-L-cysteine', 'ADP'): 1, ('gamma-L-glutamyl-L-cysteine', 'phosphate'): 1}


In [53]:
edge_weights[('ATP', 'L-cysteine')]

1

In [54]:
('ATP','L-cysteine') in edge_weights.keys()

True

In [55]:
('L-cysteine','ATP') in edge_weights.keys()

True

In [56]:
# Testing one of the setups
import numpy as np
a = np.array([1,2,3])
b = np.array([1,2,3])
print (np.multiply(a,b))

[1 4 9]


In [57]:
# Helper functions
import numpy as np
def retrieve_edge_wgt(v1, v2, weight):
    """ to handle the directionality of node pair """
    if (v1, v2) in weight.keys():
        return weight[(v1, v2)]
    else:
        return weight[(v2, v1)]

def get_Weighted_Neighbours(G, node):
    """ 
    Generate a list of neighbours of 
    a node with corresponding probability 
    """
    node_weights = nx.get_node_attributes(G, "weight")
    
    
    # Prepare neighbour list and obtain weight of each attribute for nodes and edges
    edge_wgts = nx.get_edge_attributes(G, "weight")
    node_neighbours = [adj_node for adj_node, datadict in G[node].items()]
    self_node_wgt = np.array(len(node_neighbours))*node_weights[node]
    # print (self_node_wgt)
    neighbour_wgts = [datadict['weight'] for adj_node, datadict in G[node].items()]
    neighbour_pair_wgts = list()
    for neigbhour in node_neighbours:
        neighbour_pair_wgts.append(edge_wgts[(node, neigbhour)])
    
    # Calculate the transition probability from start node to respective neighbour
    elementwise_wgts = np.multiply(self_node_wgt, np.array(neighbour_wgts), np.array(neighbour_pair_wgts))
    total_weight = sum(elementwise_wgts)
    neighbour_prob = [weight/total_weight for weight in elementwise_wgts]
    
    return node_neighbours, neighbour_prob

In [58]:
neighbour_list, neighbour_prob = get_Weighted_Neighbours(G, 'L-glutamate')

In [59]:
print (neighbour_list)
print (neighbour_prob)

['catalyses']
[1.0]


In [60]:
# Obtain next node
import random

def get_nextNode(G, start_node):
    """ 
    obtain next node of a given start_node
    """
    neighbour_list, neighbour_prob = get_Weighted_Neighbours(G, start_node)
    next_node = random.choices(neighbour_list, neighbour_prob)
    return next_node

In [61]:
next_node = get_nextNode(G, 'catalyses')

In [62]:
print (next_node)

['ATP']


In [63]:
def get_random_walk(graph, start_node, n_steps:10):
    """ 
    Given a graph and a node, 
    return a random walk starting from the node 
    """
    local_path = [start_node]
    node = start_node
    for i in range(n_steps):
        next_node = get_nextNode(G, node)
        local_path.append(next_node[0])
        node = next_node[0]
    return local_path

In [64]:
# Testing
path = get_random_walk(graph=G, start_node='L-glutamate', n_steps=10)

In [65]:
print (path)

['L-glutamate', 'catalyses', 'L-glutamate', 'catalyses', 'ATP', 'catalyses', 'produce', 'ADP', 'produce', 'catalyses', 'L-glutamate']


In [66]:
# Construct a list collecting these random walks
nodes = [G.nodes]
print (nodes)

[NodeView(('L-glutamate', 'catalyses', 'ATP', 'and', 'L-cysteine', 'to', 'produce', 'ADP', 'phosphate', 'gamma-L-glutamyl-L-cysteine'))]


In [67]:
path_collection = list()
for node in tqdm(G.nodes):
    for i in range(1000):
        path = get_random_walk(graph=G, start_node=node, n_steps=200)
        path_collection.append(path)

100%|██████████| 10/10 [01:08<00:00,  6.85s/it]


In [68]:
print (path_collection[0])

['L-glutamate', 'catalyses', 'produce', 'ADP', 'gamma-L-glutamyl-L-cysteine', 'ADP', 'produce', 'ADP', 'produce', 'ADP', 'produce', 'ADP', 'produce', 'catalyses', 'produce', 'ADP', 'and', 'phosphate', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'ADP', 'produce', 'to', 'produce', 'ADP', 'produce', 'ADP', 'produce', 'ADP', 'and', 'gamma-L-glutamyl-L-cysteine', 'ADP', 'produce', 'catalyses', 'ATP', 'and', 'ATP', 'catalyses', 'ATP', 'catalyses', 'ATP', 'catalyses', 'L-glutamate', 'catalyses', 'ATP', 'catalyses', 'ATP', 'catalyses', 'ATP', 'and', 'gamma-L-glutamyl-L-cysteine', 'ADP', 'and', 'phosphate', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'ADP', 'and', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'gamma-L-glutamyl-L-cysteine', 'ADP', 'produce', 'ADP', 'produce', 'ADP', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'gamma-L-glutamyl-L-cysteine', 'phosphate', 'ADP', 'produce', 'to', 'produce', 'ADP', 'and', 'gamma-L-glutamyl-L-cysteine', 'phosp

In [69]:
len(path_collection)

10000

In [70]:
df = pd.DataFrame(index=range(len(path_collection)), columns=['path'])

In [71]:
for i in range(len(path_collection)):
    df.loc[i, 'path'] = path_collection[i]

In [72]:
df.head()

,path
0,"[L-glutamate, catalyses, produce, ADP, gamma-L..."
1,"[L-glutamate, catalyses, ATP, catalyses, L-glu..."
2,"[L-glutamate, catalyses, ATP, L-cysteine, ATP,..."
3,"[L-glutamate, catalyses, produce, ADP, phospha..."
4,"[L-glutamate, catalyses, produce, ADP, gamma-L..."


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10000 non-null  object
dtypes: object(1)
memory usage: 78.2+ KB


In [74]:
df.to_csv(PATH+"RandomWalk.csv",index=False)